In this notebook we will call the model that we have deployed in **05_customer_churn_deploy_model**. We will first use the Azure ML SDK and afterwards the Python request package to make an HTTP call to the model API. We will see that authentication is required as we have specified this during model deployment.

# Notebook Setup

In [1]:
## Import libraries
# Standard libraries
import json
import requests

# Azure ML SDK libraries
import azureml.core
from azureml.core import Model, Workspace, Webservice

# Load Workspace

In [2]:
# Load the workspace from the saved config file
ws = Workspace.from_config()
print("Ready to use Azure ML {} to work with {}".format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.0.85 to work with sbazuremlws


# Use Model with Azure ML SDK

In [3]:
# Retrieve service where model is deployed
service_name = 'churn-prediction-service-1'
service = Webservice(ws, service_name)

In [4]:
# This is our new customer data point. We want to predict whether this customer churns or not.
x_new = ["1111-TESTC","Female",0,1,0,28,1,1,"Fiber optic",0,0,1,1,1,1,"Month-to-month",1,"Electronic check",104.8,3046.05]
print ('Customer with ID: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class and probabilities
predicted_class = json.loads(predictions)
print(predicted_class)

Customer with ID: 1111-TESTC
{'predicted_classes': ['Churn'], 'predicted_classes_probas': [[0.24526296651316704, 0.754737033486833]]}


# Use Model with HTTP request

In [6]:
# Get web service endpoint (can be retrieved from the workspace also)
endpoint = service.scoring_uri
print(endpoint)

http://40.114.250.46:80/api/v1/service/churn-prediction-service-1/score


In [20]:
# First try without authentication
import requests
import json

x_new = ["1111-TESTC","Female",0,1,0,28,1,1,"Fiber optic",0,0,1,1,1,1,"Month-to-month",1,"Electronic check",104.8,3046.05]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

print ("Customer {}".format(x_new), predicted_classes)

### A JSONDecodeError is expected here! We need to authorize!

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [15]:
# if (key) auth is enabled, retrieve the API keys. AML generates two keys.
# key1, key2 = aks_service.get_keys()
# print(key1)

# Retrieve token for authentication
access_token, refresh_after = service.get_token()

In [16]:
# If token auth is enabled, don't forget to add token to the HTTP header.
headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + access_token}

In [19]:
x_new = ["1111-TESTC","Female",0,1,0,28,1,1,"Fiber optic",0,0,1,1,1,1,"Month-to-month",1,"Electronic check",104.8,3046.05]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# # If token auth is enabled, don't forget to add token to the HTTP header.
headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + access_token}

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

print ("Customer {}".format(x_new), predicted_classes)

Customer ['1111-TESTC', 'Female', 0, 1, 0, 28, 1, 1, 'Fiber optic', 0, 0, 1, 1, 1, 1, 'Month-to-month', 1, 'Electronic check', 104.8, 3046.05] {'predicted_classes': ['Churn'], 'predicted_classes_probas': [[0.24526296651316704, 0.754737033486833]]}
